In [22]:
import pynq
import time
import numpy as np
import socket

In [23]:
HOST = "127.0.0.2"  # The server's hostname or IP address
PORT = 65432  # The port used by the server

In [21]:
bitstream = "/nethome/djohri7/Desktop/board_run/Lab3/partA/optimized/real_matmul_optimized.bit"
overlay = pynq.Overlay(bitstream)
dut = overlay.real_matmul_0

TimeoutError: [Errno 110] Connection timed out

In [17]:
A = pynq.allocate((100, 150), dtype='int16')
B = pynq.allocate((150, 200), dtype='int16')
C = pynq.allocate((100, 200), dtype='int16')

In [18]:
arr1=[]
arr2=[]
arr3=[]

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    s.connect((HOST, PORT))
    result_bytes = s.recv(4*100*150)
    result = np.frombuffer(result_bytes, dtype=np.int16)
    arr1= result.copy()
    
    result_bytes = s.recv((4*15000))
    result = np.frombuffer(result_bytes, dtype=np.int16)
    arr2.append(result)
    
    result_bytes = s.recv((4*15001))
    result = np.frombuffer(result_bytes, dtype=np.int16)
    arr3.append(result)
    
arr1 = np.array(list(arr1), dtype='int16')
arr2 = np.array(list(np.concatenate(arr2+arr3)), dtype='int16')

arr1= arr1.reshape(100,150)
arr2= arr2.reshape(150,200)

print("1st Array received: ", arr1)
print("2nd Array received: ", arr2)

ValueError: cannot reshape array of size 30000 into shape (100,150)

In [ ]:
for i in range(100):
    for j in range(150):
        A[i][j]= arr1[i][j]

for i in range(150):
    for j in range(200):
        B[i][j]= arr2[i][j]

In [ ]:
A.sync_to_device()
B.sync_to_device()
dut.register_map
dut.register_map.MatA_DRAM_1 = A.device_address & 0xFFFFFFFF
dut.register_map.MatA_DRAM_2 = A.device_address >> 32

dut.register_map.MatB_DRAM_1 = B.device_address & 0xFFFFFFFF
dut.register_map.MatB_DRAM_2 = B.device_address >> 32

dut.register_map.MatC_DRAM_1 = C.device_address & 0xFFFFFFFF
dut.register_map.MatC_DRAM_2 = C.device_address >> 32
start_time = time.time()

dut.register_map.CTRL.AP_START = 1
dut.register_map.CTRL[4] = 1
while not dut.register_map.CTRL.AP_DONE: pass

end_time = time.time()
duration = end_time - start_time
print(f'Kernel completed in {duration * 1000:.2f}ms')
C.sync_from_device()

In [ ]:
C_expected= np.matmul(arr1,arr2)

In [ ]:
passed=1
for i in range(100):
    for j in range(200):
        if(C_expected[i][j]!= C[i][j]):
            print("Mismatch at C_expected["+str(i)+"]["+str(j)+"]: Expected: "+str(C_expected[i][j])+" \t Actual: "+str(C[i][j])+"\n")
            passed=0

if(passed):
    print("-----------------------------------\n");
    print("|         TEST PASSED!            |\n");
    print("-----------------------------------\n");

else:
    print("-----------------------------------\n");
    print("|         TEST FAILED :(          |\n");
    print("-----------------------------------\n");

In [ ]:
overlay.free()

In [ ]:
start_time = time.time()

C_out = np.matmul(A, B)

end_time = time.time()
duration = end_time - start_time
print(f'PS computation completed in {duration * 1000:.2f}ms')

In [ ]:
#Sending the data back
c= np.empty(shape=(100,200))
for i in range(100):
    for j in range(200):
        c[i][j]= C[i][j]

c_ser= c.flatten()
c_1_bytes = np.array(c_ser[0:10000], dtype=np.int16).tobytes()
c_2_bytes = np.array(c_ser[10000:20000], dtype=np.int16).tobytes()

print(c_ser[0:10000])
print(c_ser[10000:20000])
print(c_ser)


PORT = 65433
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    s.connect((HOST, PORT))

    data= c_1_bytes
    s.sendall(data)
            
    data= c_2_bytes
    s.sendall(data)
  